In [1]:
import requests  # Importation du module requests pour faire des requêtes HTTP
import datetime  # Importation du module datetime pour travailler avec des dates et des heures

# URL de base de l'API OpenWeatherMap pour obtenir les données météorologiques
base_url = "http://api.openweathermap.org/data/2.5/forecast"

# Votre clé API pour accéder à l'API OpenWeatherMap (remplacez-la par la vôtre)
api_key = 'XXX'

# Nom de la ville pour laquelle on souhaite obtenir les données météorologiques
city_name = 'Paris'

# Paramètres de la requête : nom de la ville, clé API et unités métriques pour les données
params = {
    'q': city_name,
    'appid': api_key,
    'units': 'metric',  # Pour obtenir les données en unités métriques
    'cnt': 40  # Limite le nombre de données (5 jours x 8 prévisions par jour)
}

# Envoi de la requête GET à l'API OpenWeatherMap avec les paramètres spécifiés
response = requests.get(base_url, params=params)

# Vérification du statut de la réponse pour s'assurer qu'elle est réussie (code 200)
if response.status_code == 200:
    # Conversion de la réponse JSON en dictionnaire Python
    data = response.json()
    forecast_list = data['list']
    city = data['city']['name']
    
    weather_data = []
    
    # Extraction des informations principales de la réponse
    for forecast in forecast_list:
        main = forecast['main']
        weather = forecast['weather'][0]
        wind = forecast['wind']
        localtime = datetime.datetime.utcfromtimestamp(forecast['dt']) + datetime.timedelta(seconds=data['city']['timezone'])
    
    # Création d'un dictionnaire contenant les données météorologiques actuelles
    forecast_data = {
        'city': data['city']['name'],              # Nom de la ville
        'temperature': main['temp'],       # Température actuelle
        'temperature_min': main['temp_min'],  # Température minimale
        'temperature_max': main['temp_max'],  # Température maximale
        'pressure': main['pressure'],      # Pression atmosphérique
        'humidity': main['humidity'],      # Humidité
        'weather': weather['description'], # Description du temps (ex : "clear sky")
        'wind_speed': wind['speed'],  # Vitesse du vent
        'local_time':  localtime.strftime('%Y-%m-%d %H:%M:%S')  # Formater la date et l'heure au format SQLite
    }
else:
    # En cas d'erreur dans la requête, stocker un message d'erreur
    forecast_data = {"error": "Unable to fetch data"}

In [2]:
forecast_data

{'city': 'Paris',
 'temperature': 17.18,
 'temperature_min': 17.18,
 'temperature_max': 17.18,
 'pressure': 1019,
 'humidity': 79,
 'weather': 'broken clouds',
 'wind_speed': 3.6,
 'local_time': '2024-07-23 08:00:00'}

In [3]:
import sqlite3  # Importation du module sqlite3 pour travailler avec des bases de données SQLite

# Nom de la table
db_name = 'weather.db'

# Nom de la table
table_name = 'forecast'

if 'error' not in forecast_data:
    # Établir une connexion à la base de données SQLite nommée 'weather.db'
    # Si cette base de données n'existe pas, elle sera créée dans le répertoire courant
    conn = sqlite3.connect(db_name)
    
    # Création d'un curseur pour permettre l'exécution de commandes SQL
    cursor = conn.cursor()
    
    cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    
    # Exécution d'une commande SQL pour créer une table 'current' si elle n'existe pas déjà
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {table_name} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            city TEXT,
            temperature REAL,
            temperature_min REAL,
            temperature_max REAL,
            pressure INTEGER,
            humidity INTEGER,
            weather TEXT,
            wind_speed REAL,
            local_time TEXT,
            UNIQUE(city, local_time)
        )
    ''')
    
    # Valider la transaction en cours et appliquer les modifications
    conn.commit()
    
    # Fermer la connexion à la base de données pour libérer les ressources
    conn.close()
    
    # Affichage d'un message 
    print(f"La table {table_name} a été créée avec succès.")
else:
    print(current_weather_data['error'])

La table forecast a été créée avec succès.


In [4]:
# Connexion à la base de données SQLite nommée 'weather_data.db'
# Si cette base de données n'existe pas, elle sera créée
conn = sqlite3.connect(db_name)

# Création d'un curseur permettant d'exécuter des commandes SQL
cursor = conn.cursor()

# Exécution d'une commande SQL pour insérer des données dans la table 'weather'
# Les valeurs à insérer sont spécifiées sous forme de paramètres pour éviter les injections SQL
cursor.execute(f'''
    INSERT INTO {table_name} (city, temperature, temperature_min, temperature_max, pressure, humidity, weather, wind_speed, local_time)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
''', (forecast_data['city'], forecast_data['temperature'], forecast_data['temperature_min'], forecast_data['temperature_max'], forecast_data['pressure'], forecast_data['humidity'], forecast_data['weather'], forecast_data['wind_speed'], forecast_data['local_time']))

# Validation des modifications apportées à la base de données
conn.commit()

# Fermeture de la connexion à la base de données
conn.close()

print(f"Données insérées avec succès dans la table {table_name} de la base de données SQLite {db_name}.")

Données insérées avec succès dans la table forecast de la base de données SQLite weather.db.


In [6]:
# Connexion à la base de données SQLite
# La variable `db_name` doit contenir le nom de la base de données SQLite (par exemple, 'weather_data.db')
conn = sqlite3.connect(db_name)

# Création d'un curseur pour permettre l'exécution de commandes SQL
cursor = conn.cursor()

# Exécution d'une requête SQL pour récupérer toutes les lignes de la table `forecast`
# La variable `table_name` doit contenir le nom de la table à interroger (par exemple, 'weather')
cursor.execute(f"SELECT * FROM {table_name}")

# Récupérer toutes les lignes de résultats de la requête exécutée
rows = cursor.fetchall()

# Afficher les résultats récupérés
for row in rows:
    print(row)

# Fermer la connexion à la base de données pour libérer les ressources
conn.close()

(1, 'Paris', 17.18, 17.18, 17.18, 1019, 79, 'broken clouds', 3.6, '2024-07-23 08:00:00')
